In [110]:
import pandas as pd
import logging
import json
from itertools import product
from datetime import datetime, timedelta

pd.set_option("display.float_format", "{:.2f}".format)

In [111]:
# Functions

def get_logger() -> logging.Logger:
    """
    Get logger instance.
    """
    level = logging.INFO
    logger = logging.getLogger(__name__)
    logger.setLevel(level)
    logger.propagate = False
    logger.handlers = []
    console_handler = logging.StreamHandler()
    console_handler.setLevel(level)
    formatter = logging.Formatter(
        "%(asctime)s %(levelname)s | %(message)s", "%Y-%m-%d %H:%M:%S"
    )
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    return logger

def json_expansion(df, payload_column):
  expanded_list = []
  os_multiplier = []
  for idx, row in df[payload_column].items():
    row_update = json.loads(row)['allocations']
    row_os_multiplier = json.loads(row)['os_multiplier']

    expanded = pd.json_normalize(row_update)
    expanded['original_index'] = idx
    expanded['os_multiplier'] = row_os_multiplier

    expanded_list.append(expanded)

  return expanded_list

def cap_and_normalise(df, column):
  total_allocation = df[column].sum()
  adjusted_allocation = 1
  capped_list = []

  for index, row in df.iterrows():
      capped_allocation = min((row[column] * adjusted_allocation)/total_allocation, 0.05)
      total_allocation = total_allocation  - row[column]
      adjusted_allocation = adjusted_allocation - capped_allocation
      capped_list.append(capped_allocation*10000000)

  return capped_list

In [112]:
# Data ingestion

ballot_df = pd.read_csv("/content/Voting data export final.csv")
badgeholders_df = pd.read_csv("/content/RPGF4_badgeholders.csv")
project_metrics_df = pd.read_csv("/content/op_rf4_impact_metrics_by_project (1).csv")

In [113]:
# Vote and voter validation

log = get_logger()

# Check if voter_address is unique
if ballot_df["Address"].nunique() == ballot_df.shape[0]:
    log.info("Check - Address is unique.")
else:
    diff = ballot_df.shape[0] - ballot_df["Address"].nunique()
    log.info(f"Check - Address is not unique. There are {diff} duplicates.")

# Check if all address in df are in df_badge_holders
voter_validity_check = ballot_df["Address"].str.lower().isin(badgeholders_df["Badgeholder"].str.lower())

if ballot_df[voter_validity_check].shape[0] == ballot_df.shape[0]:
    log.info("Check - All addresses in df are in df_badge_holders.")
else:
    diff = ballot_df.shape[0] - ballot_df[voter_validity_check].shape[0]
    log.info(f"Check - {diff} addresses in voting are not Badge Holders.")
    # print those addresses not in df_badge_holders
    display(ballot_df[voter_validity_check])

# Check if all ballots have a signature
ballot_bh_df = ballot_df[voter_validity_check]
signature_filtered_df = ballot_bh_df[ballot_bh_df['Signature'].notna()]

2024-07-17 13:42:36 INFO | Check - Address is unique.
2024-07-17 13:42:36 INFO | Check - All addresses in df are in df_badge_holders.


In [114]:
expanded_list = json_expansion(signature_filtered_df, 'Payload')
expanded_df = pd.concat(expanded_list, ignore_index=True)

result_df = expanded_df.set_index("original_index").join(ballot_df.set_index(ballot_df.index))

# Define the columns and aggregations
sum_cols = list(expanded_df.columns)
sum_cols.remove('os_multiplier')
sum_cols.remove("original_index")

unique_cols = list(ballot_df.columns) + ['os_multiplier']

# Define the aggregations
aggregations = {
    col: 'sum' for col in sum_cols
}
aggregations.update({
    col: lambda x: x.iloc[0] for col in unique_cols
})

# Group by index and use agg() to combine the values
grouped_result = result_df.groupby(result_df.index).agg(aggregations)
grouped_result['metric_total'] = grouped_result[sum_cols].sum(axis=1)

In [115]:
# Generate ImpactMetricShares by Project and Badgeholder

bh_subset = signature_filtered_df[['Address', 'Payload']]

expanded_list = []
expanded_index = []
for idx, row in bh_subset["Payload"].items():

  row_update = json.loads(row)['os_multiplier']
  expanded_list.append(row_update)
  expanded_index.append(idx)

multiplier_expanded = pd.DataFrame(
    {'original_index': expanded_index,
     'os_multiplier': expanded_list
    })

bh_subset = multiplier_expanded.set_index("original_index").join(bh_subset.set_index(bh_subset.index))

# Create a list of all possible combinations of indices
indices = list(product(project_metrics_df.index, bh_subset.index))

# Create a DataFrame with the combined indices
combined_df = pd.DataFrame(indices, columns=['index1', 'index2'])

# Perform the join using merge()
metric_sum_df = pd.merge(combined_df, project_metrics_df, left_on='index1', right_index=True) \
          .merge(bh_subset, left_on='index2', right_index=True)

for col in sum_cols:
    metric_sum_df.loc[metric_sum_df['is_oss'] == True, col] = \
    metric_sum_df.loc[metric_sum_df['is_oss'] == True, col] * metric_sum_df.loc[metric_sum_df['is_oss'] == True, 'os_multiplier']

for address in bh_subset['Address']:
    for col in sum_cols:
        metric_sum_df.loc[metric_sum_df['Address'] == address, col] = \
        metric_sum_df.loc[metric_sum_df['Address'] == address, col] / metric_sum_df.loc[metric_sum_df['Address'] == address, col].sum()

In [116]:
# Generate the Score by Badgeholder

badgeholder_allocation_df = metric_sum_df.copy()

for address in bh_subset['Address']:

    for col in sum_cols:
        if grouped_result.loc[grouped_result['Address'] == address, col].iloc[0] == 0:
          badgeholder_allocation_df.loc[badgeholder_allocation_df['Address'] == address, col] = badgeholder_allocation_df.loc[badgeholder_allocation_df['Address'] == address, col]* 0
        else:
          badgeholder_allocation_df.loc[badgeholder_allocation_df['Address'] == address, col] = badgeholder_allocation_df.loc[badgeholder_allocation_df['Address'] == address, col]* (grouped_result.loc[grouped_result['Address'] == address, col].iloc[0]/100)

# Calculate badgeholder score for each project
badgeholder_allocation_df['badgeholder_allocation'] = badgeholder_allocation_df[sum_cols].sum(axis=1)

In [117]:
# Median allocations per project

median_score_df = badgeholder_allocation_df.copy()

updated_median_df = pd.DataFrame()
capped_list = []

for address in bh_subset['Address']:
  df = median_score_df.loc[median_score_df['Address'] == address].sort_values(by ='badgeholder_allocation', ascending=False)
  updated_median_df = pd.concat([updated_median_df, df], ignore_index=True)

  # Normalise and re-cap score values
  if not capped_list:
    capped_list = cap_and_normalise(df,'badgeholder_allocation')
  else:
    capped_list = capped_list + cap_and_normalise(df,'badgeholder_allocation')

updated_median_df['badgeholder_score'] = capped_list
median_scores = updated_median_df.groupby(['project_name','application_id']).agg(median_score=('badgeholder_score', 'median'))

In [118]:
# Generate final allocation by applying cap and normalizing to 10M

# Normalise and re-cap score values
final_median_scores = median_scores.copy().sort_values(by ='median_score', ascending=False)
final_median_scores['adjusted_score'] = cap_and_normalise(final_median_scores, 'median_score')

# Filter out projects with a badgeholder_score of less than 1000
filtered_median_scores = final_median_scores[final_median_scores['adjusted_score'] > 1000].sort_values(by ='adjusted_score', ascending=False)
filtered_median_scores['final_score'] = cap_and_normalise(filtered_median_scores, 'adjusted_score')

# Round score columns to 1 decimal place
cols_to_round = ['median_score', 'adjusted_score', 'final_score']
for col in cols_to_round:
    filtered_median_scores[col] = filtered_median_scores[col].round(1).astype(object)

filtered_median_scores = filtered_median_scores.sort_values(by = 'final_score', ascending=False)
filtered_median_scores.to_csv("rpgf4_final_score.csv", encoding='utf-8')
filtered_median_scores

,,median_score,adjusted_score,final_score
project_name,application_id,,,
Zora,0x9102357674825ed224734387fbefbf41c16fe5d9764c2f7f3e897ad3325d3990,500000.00,500000.00,500000.00
Layer3,0x91a4420e2fcc8311e97dad480f201a8ce221f2cd64c2de77280cbcc6ce193752,500000.00,500000.00,500000.00
LI.FI,0x517eaa9c56951de89261f2d7830ea49aae92f2a903104a17d9c5c2edd4959806,440940.50,480922.70,481556.20
Stargate Finance,0x62e37e96aa6e1cbfb6bd24b97c4b8f1e12cc3fe35d5388d2f041c42a12b40745,390718.50,426146.80,426708.10
ODOS,0xd5b4c54b12bf86f6eb67fec81032809a16ff1c4b4c7f0d5898fc86367db86ca6,328804.50,358618.80,359091.20
...,...,...,...,...
IntentX,0x2ddf5b9dc64f873d6557b4f63de936cf278f8851deda857244a1be44d1a1e950,1155.60,1260.40,1262.00
Egg Wars: A Game on Base,0xa4d3729a488eeabcc3a59de70b760391db50e9bf488d5942e704089c58825c24,1140.40,1243.90,1245.50
BaseDoge,0x69e36f30f2a21afb2eb61156ca00f303434ba8868315abec3eb8780b812fe976,1067.40,1164.20,1165.70
